In [2]:
import numpy as np
import os
import tensorflow as tf
import pandas as pd
import glob
import math
import matplotlib.pyplot as plt
from PIL import Image
from collections import Counter
from torch.utils.data import (
    Dataset,
    DataLoader
)
import torch
import torchvision.transforms as transforms
from skimage import io

from transformers import ConvNextFeatureExtractor, ConvNextForImageClassification

In [3]:
basedir = os.getcwd()
datadir = basedir + '/data'
# reading the csv file with annotated image file names
test_cultivar= pd.read_csv(datadir + '/sample_submission.csv')
test_cultivar.dropna(inplace=True)

# creating list of unique cultivars
labels=list(np.unique(test_cultivar['cultivar']))

# turning cultivar labels into strings
test_cultivar['cultivar']=test_cultivar['cultivar'].astype(str)
test_cultivar["cultivar_index"] = test_cultivar["cultivar"].map(lambda item: labels.index(item))

test_cultivar

,filename,cultivar,cultivar_index
0,1000005362.png,PI_152923,0
1,1000099707.png,PI_152923,0
2,1000135300.png,PI_152923,0
3,1000136796.png,PI_152923,0
4,1000292439.png,PI_152923,0
...,...,...,...
23634,999578153.png,PI_152923,0
23635,999692877.png,PI_152923,0
23636,999756998.png,PI_152923,0
23637,999892248.png,PI_152923,0


In [125]:
model_name_or_path = basedir + '/results/checkpoint-300'
feature_extractor = ConvNextFeatureExtractor.from_pretrained(model_name_or_path)

# building feature extractor to grab labels
class FeatureExtractor(object):
    def __call__(self, image, target):
        sample = feature_extractor(image, return_tensors='pt')
        sample["labels"] = target
        return sample

class CultivarDataset(Dataset):
    def __init__(self, df_img, df_label, transform):
        self.labels = df_label
        self.image_path = df_img
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image_path = datadir + '/test/' + self.image_path[idx]
        image = io.imread(image_path)
        
        y_label = torch.tensor(int(self.labels.iloc[idx]))
       # y_label = int(self.labels.iloc[idx])
        
        data = self.transform(image,y_label)
        data['pixel_values'] = torch.squeeze(data['pixel_values'])
        return data

loading feature extractor configuration file /home/maymap/results/checkpoint-300/preprocessor_config.json
Feature extractor ConvNextFeatureExtractor {
  "crop_pct": 0.875,
  "do_normalize": true,
  "do_resize": true,
  "feature_extractor_type": "ConvNextFeatureExtractor",
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "size": 224
}



In [126]:
test_ds = CultivarDataset(
    df_img = test_cultivar['filename'],
    df_label = test_cultivar['cultivar_index'],
    transform = FeatureExtractor())

In [127]:
test_ds[0]

{'pixel_values': tensor([[[-1.0733, -0.9363, -0.9705,  ..., -1.1589, -1.1418, -1.1589],
         [-0.9705, -0.9363, -0.9534,  ..., -1.1589, -1.1760, -1.1760],
         [-1.0390, -1.0390, -0.8335,  ..., -1.0562, -1.0904, -1.1075],
         ...,
         [-0.9705, -1.0219, -1.1247,  ..., -0.6452, -0.6794, -0.8678],
         [-1.1418, -1.1760, -1.2617,  ..., -0.7308, -0.9877, -1.0562],
         [-1.3130, -1.3130, -1.3815,  ...,  0.2796,  0.1426,  0.1768]],

        [[-0.9503, -0.8277, -0.8277,  ..., -0.9853, -0.9853, -1.0028],
         [-0.8627, -0.7927, -0.8277,  ..., -1.0378, -1.0378, -1.0553],
         [-0.9328, -0.9503, -0.7402,  ..., -0.8978, -0.9328, -0.9678],
         ...,
         [-0.7752, -0.8102, -0.9153,  ..., -0.3901, -0.5126, -0.7052],
         [-1.0028, -1.0553, -1.1253,  ..., -0.5651, -0.7927, -0.8277],
         [-1.1779, -1.2304, -1.2654,  ...,  0.4678,  0.3627,  0.3803]],

        [[-1.0027, -0.9156, -0.8807,  ..., -1.0027, -0.9678, -1.0027],
         [-0.9156, -0.8110, 

In [129]:
(test_ds.__getitem__(9)['pixel_values']).shape

torch.Size([3, 224, 224])

In [ ]:
from transformers import Trainer
from transformers import TrainingArguments
model = ConvNextForImageClassification.from_pretrained(model_name_or_path)
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'][0] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)
training_args = TrainingArguments(
  output_dir="./results",
  logging_dir = '/home/runs',
  evaluation_strategy='steps',
  per_device_train_batch_size=64,
  num_train_epochs=4,
  save_total_limit = 4, # Only last 4 models are saved. Older ones are deleted.
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  load_best_model_at_end=True,
)
trainer = trainer = Trainer(
    model=model
)
trainer.predict(test_ds)


loading configuration file /home/maymap/results/checkpoint-300/config.json
Model config ConvNextConfig {
  "_name_or_path": "facebook/convnext-tiny-224",
  "architectures": [
    "ConvNextForImageClassification"
  ],
  "depths": [
    3,
    3,
    9,
    3
  ],
  "drop_path_rate": 0.0,
  "hidden_act": "gelu",
  "hidden_sizes": [
    96,
    192,
    384,
    768
  ],
  "id2label": {
    "0": "PI_144134",
    "1": "PI_145619",
    "2": "PI_145626",
    "3": "PI_145633",
    "4": "PI_146890",
    "5": "PI_152591",
    "6": "PI_152651",
    "7": "PI_152694",
    "8": "PI_152727",
    "9": "PI_152728",
    "10": "PI_152730",
    "11": "PI_152733",
    "12": "PI_152751",
    "13": "PI_152771",
    "14": "PI_152816",
    "15": "PI_152828",
    "16": "PI_152860",
    "17": "PI_152862",
    "18": "PI_152923",
    "19": "PI_152961",
    "20": "PI_152965",
    "21": "PI_152966",
    "22": "PI_152967",
    "23": "PI_152971",
    "24": "PI_153877",
    "25": "PI_154750",
    "26": "PI_154844",
  